# Segment 2 Lab 1

Linear regression, generalization and overfitting

## Disclaimer again!

I will go through this quite fast - if you're new to ML, use this to gain intuition!

In [ ]:
# imports from incredibly common packages: numpy, matplotlib, sci-kit learn

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# To set up our problem

## We have 100 Home Appliances.

Each of them have a **price** and a **height**.

Unknown to us - all the home appliances have a price that is (strangely) closely related to its height!

The price is actually closely given by:
$$
price = 200 + 100 \cdot height - 40 \cdot height^2 + 4 \cdot height^3
$$

This means that if we were to create 3 features:
1. height
2. height squared
3. height cubed

Then a linear regression model would be able to predict the price closely.

Let's see how this works.

In [ ]:
# Generate synthetic data for Home Appliances: 100 points with prices that roughly follow a cubic curve

np.random.seed(42)
n_samples = 100
heights = np.random.uniform(0, 10, n_samples)
noise = np.random.normal(0, 50, n_samples)
prices = 200 + 100*heights - 40*heights**2 + 4*heights**3 + noise

In [ ]:
# Plot the generated data

plt.figure(figsize=(10, 6))
plt.scatter(heights, prices, alpha=0.5)
plt.title('Home Appliance Price vs Height')
plt.xlabel('Height')
plt.ylabel('Price')
plt.show()

In [ ]:
# Every numpy array has a shape that represents the dimensions of the array
# Our weights are currently a one dimensional list of floats:

print(heights.shape)
print(heights)

In [ ]:
# We want to turn this into a 2D matrix, with a row for each of our 100 datapoints
# And columns for each feature (there is only 1)

X = heights.reshape(-1, 1)
print(X.shape)
print(X)

In [ ]:
# We can use this utility method from sk-learn to split up our data into train and test:

X_train, X_test, y_train, y_test = train_test_split(X, prices, test_size=0.2, random_state=42)

In [ ]:
# Running Linear Regression is simply a couple of lines of code - predict price based only on height:

model = LinearRegression()
model.fit(X_train, y_train)

# Making predictions

Now we have a trained model and we can make predictions. We will run the model on our test dataset.

We'll calculate 2 key metrics: Mean Squared Error and R squared.

**Mean Squared Error** as it sounds, the average of the square of the difference between y and y hat

**R squared** measures how well the predictions fit the truth;  
0 means that the model explains none of the variability and is no better than always guessing the mean value   
100% means it fully explains it  
A negative number suggests that the model is worse than just guessing the average!

In [ ]:
# Run predictions on test dataset using model.predict

y_hat_test = model.predict(X_test)
mse = mean_squared_error(y_test, y_hat_test)
r2 = r2_score(y_test, y_hat_test)

In [ ]:
# Let's visualize the results

plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test, alpha=0.5)
plt.plot(X_test, y_hat_test, color='red', linewidth=2)
plt.title(f'Linear Regression with 1 feature (height) - performance on unseen Test data\nMSE: {mse:,.0f}\nR squared: {r2*100:.0f}%')
plt.xlabel('Height')
plt.ylabel('Price')
plt.show()

In [ ]:
# Now let's add a SECOND feature to our input data, in addition to the height
# Let's add a feature which is the SQUARE of the height

X2 = np.column_stack((heights, heights**2))
print(X2.shape)
print(X2)

In [ ]:
# Split into train and test and run Linear Regression

X_train, X_test, y_train, y_test = train_test_split(X2, prices, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_hat_test = model.predict(X_test)
mse = mean_squared_error(y_test, y_hat_test)
r2 = r2_score(y_test, y_hat_test)

In [ ]:
# Sort X_test for smooth curve plotting

X_test_sorted = X_test[X_test[:, 0].argsort()]
y_hat_test_sorted = model.predict(X_test_sorted)

In [ ]:
# Plot the results

plt.figure(figsize=(10, 6))
plt.scatter(X_test[:, 0], y_test, alpha=0.5)
plt.plot(X_test_sorted[:, 0], y_hat_test_sorted, color='red', linewidth=2)
plt.title(f'Linear Regression - performance on unseen Test data\nMSE: {mse:,.0f}\nR squared: {r2*100:.0f}%')
plt.xlabel('Height')
plt.ylabel('Price')
plt.show()

In [ ]:
# Step 3: Polynomial Regression (Degree 3)

X3 = np.column_stack((heights, heights**2, heights**3))
X_train, X_test, y_train, y_test = train_test_split(X3, prices, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_hat_test = model.predict(X_test)
mse = mean_squared_error(y_test, y_hat_test)
r2 = r2_score(y_test, y_hat_test)

In [ ]:
# Sort X_test for smooth curve plotting

X_test_sorted = X_test[X_test[:, 0].argsort()]
y_hat_test_sorted = model.predict(X_test_sorted)

In [ ]:
# Visualize the results

plt.figure(figsize=(10, 6))
plt.scatter(X_test[:, 0], y_test, alpha=0.5)
plt.plot(X_test_sorted[:, 0], y_hat_test_sorted, color='red', linewidth=2)
plt.title(f'Linear Regression - performance on unseen Test data\nMSE: {mse:,.0f}\nR squared: {r2*100:.0f}%')
plt.xlabel('Height')
plt.ylabel('Price')
plt.show()

Remember the true formula is: $price = 200 + 100 \cdot height - 40 \cdot height^2 + 4 \cdot height^3$

In [ ]:
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

# Conclusion

When we give our Linear Regression model three features:

height  
height squared  
height cubed  

It's able to fit and then it generalizes well, because the results look great on unseen data.

And this makes sense because secretly, we generated data based on exactly those features.

# AND NOW... to make this come to life

We're going to make 376 features.

So many features, that our model can PERFECTLY predict every item in the Training data.

In [ ]:
def make_overfit_features(
    x,
    n_poly=30,                 # x^1 ... x^n_poly
    frac_feats=True,           # sqrt, cbrt, log, reciprocal
    n_fourier=20,              # sin/cos pairs at different frequencies
    n_hinges=20,               # ReLU hinges at many cutpoints
    n_steps=10,                # hard indicator steps
    n_rbf=30,                  # local Gaussian bumps
    n_noise=60,                # pure noise columns
    n_x_noise=20,              # x * noise interactions
    n_dupes=10,                # duplicate some columns with tiny jitter
    seed=0
):
    rng = np.random.default_rng(seed)
    x = np.asarray(x).reshape(-1)             
    n = x.shape[0]

    x_mean, x_std = x.mean(), (x.std() or 1.0)
    z = (x - x_mean) / x_std

    cols, names = [], []

    # 1) Polynomials (mix scaled & unscaled to create collinearity)
    for k in range(1, n_poly + 1):
        cols.append(z**k); names.append(f"z^{k}")
    for k in range(1, min(n_poly, 12) + 1):  # a few unscaled high-variance terms
        cols.append(x**k); names.append(f"x^{k}(raw)")

    # 2) Fractional/odd transforms
    if frac_feats:
        eps = 1e-6
        cols += [np.sqrt(np.clip(x, 0, None)), np.cbrt(np.clip(x, -1e9, None))]
        names += ["sqrt(x)", "cbrt(x)"]
        cols += [np.log(np.abs(x) + 1.0), 1.0 / (np.abs(x) + 1.0 + eps)]
        names += ["log(|x|+1)", "1/(|x|+1)"]

    # 3) Fourier basis (varied frequencies)
    # Use frequencies up to Nyquist-ish relative to the spread of x
    x_span = max(x.max() - x.min(), 1.0)
    base = np.pi / max(x_span, 1e-6)
    for j in range(1, n_fourier + 1):
        w = j * base
        cols.append(np.sin(w * x)); names.append(f"sin({j})")
        cols.append(np.cos(w * x)); names.append(f"cos({j})")

    # 4) Hinges (ReLU) and Steps at random cutpoints across x
    cuts = np.quantile(x, np.linspace(0.05, 0.95, n_hinges))
    for c in cuts:
        cols.append(np.maximum(0.0, x - c)); names.append(f"(x-{c:.3g})_+")
    step_cuts = np.quantile(x, np.linspace(0.1, 0.9, n_steps))
    for c in step_cuts:
        cols.append((x > c).astype(float)); names.append(f"1[x>{c:.3g}]")

    # 5) RBF bumps with small/medium bandwidths
    centers = np.quantile(x, np.linspace(0.05, 0.95, n_rbf))
    sigmas = np.linspace(0.15, 0.35, n_rbf) * max(x_std, 1e-6)
    for c, s in zip(centers, sigmas):
        cols.append(np.exp(-0.5 * ((x - c) / (s + 1e-12))**2))
        names.append(f"rbf(c={c:.3g},s={s:.3g})")

    # 6) Pure noise features
    for j in range(n_noise):
        eta = rng.standard_normal(n)
        cols.append(eta); names.append(f"noise[{j}]")

    # 7) Interactions of x with noise (sample-specific quirks)
    for j in range(n_x_noise):
        eta = rng.standard_normal(n)
        cols.append(x * eta); names.append(f"x*noise[{j}]")

    # 8) Duplicate a few columns with tiny jitter (ill-conditioning)
    jitter_scale = 1e-4
    pick = rng.integers(0, len(cols), size=n_dupes)
    for idx in pick:
        dup = cols[idx] + jitter_scale * rng.standard_normal(n)
        cols.append(dup); names.append(f"{names[idx]}_dup")

    X = np.column_stack(cols)
    return X, names

In [ ]:
X_overfit, feat_names = make_overfit_features(heights,
                                      n_poly=40,
                                      n_fourier=30,
                                      n_hinges=30,
                                      n_rbf=40,
                                      n_noise=120,
                                      n_x_noise=40,
                                      n_dupes=20,
                                      seed=42)

In [ ]:
", ".join(feat_names)

In [ ]:
print(X_overfit.shape)
X_overfit[0]

In [ ]:
# Linear Regression

X_train, X_test, y_train, y_test = train_test_split(X_overfit, prices, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_hat_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_hat_train)
r2_train = r2_score(y_train, y_hat_train)

y_hat_test = model.predict(X_test)
mse = mean_squared_error(y_test, y_hat_test)
r2 = r2_score(y_test, y_hat_test)

In [ ]:
X_train_sorted = X_train[X_train[:, 0].argsort()]
y_hat_train_sorted = model.predict(X_train_sorted)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_train[:, 0], y_train, alpha=0.5)
plt.plot(X_train_sorted[:, 0], y_hat_train_sorted, color='red', linewidth=2)
plt.title(f'Linear Regression - performance on Training data\nMSE: {mse_train:,.0f}\nR squared: {r2_train*100:.0f}%')
plt.xlabel('Height')
plt.ylabel('Price')
plt.show()

In [ ]:
# Sort X_test for smooth curve plotting

X_test_sorted = X_test[X_test[:, 0].argsort()]
y_hat_test_sorted = model.predict(X_test_sorted)

In [ ]:
# Plot the results

plt.figure(figsize=(10, 6))
plt.scatter(X_test[:, 0], y_test, alpha=0.5)
plt.plot(X_test_sorted[:, 0], y_hat_test_sorted, color='red', linewidth=2)
plt.title(f'Linear Regression - performance on unseen Test data\nMSE: {mse:,.0f}\nR squared: {r2*100:.0f}%')
plt.xlabel('Height')
plt.ylabel('Price')
plt.show()

# Conclusion

And so you see the surprising results. We provided the Linear Regression with the same festures as before, and MORE features if it wanted them.

And something went wrong: we got worse results.

Why?

Because armed with the extra flexibility, the model was able to fit more closely to the training data.

But it was finding new patterns in the noise that DIDN'T REFLECT the underlying pattern.

So when we extended it to make new predictions, it did worse.

And that is over-fitting!